In [2]:
print("Hello, World!")

Hello, World!


In [97]:
import glob
import json
import pandas as pd

# Get list of all JSON files in the current directory
json_files = glob.glob('../foursquare-export/checkins*.json')
print(type(json_files))
# Load each JSON file into a list of dictionaries
data = []
for file in json_files:
	with open(file, 'r') as f:
		print(f"Reading {file}")
		data.append(json.load(f))

print(type(data))
print(len(data))

<class 'list'>
Reading ../foursquare-export/checkins4.json
Reading ../foursquare-export/checkins11.json
Reading ../foursquare-export/checkins8.json
Reading ../foursquare-export/checkins9.json
Reading ../foursquare-export/checkins10.json
Reading ../foursquare-export/checkins5.json
Reading ../foursquare-export/checkins2.json
Reading ../foursquare-export/checkins3.json
Reading ../foursquare-export/checkins15.json
Reading ../foursquare-export/checkins1.json
Reading ../foursquare-export/checkins14.json
Reading ../foursquare-export/checkins13.json
Reading ../foursquare-export/checkins6.json
Reading ../foursquare-export/checkins7.json
Reading ../foursquare-export/checkins12.json
<class 'list'>
15


In [ ]:
# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(columns=[
    'id', 
    'createdAt',
    'type',
    'visibility',
    'timeZoneOffset',
    'venueId',
    'venueName',
    'venueURL',
    'commentsCount',
    'shout'
])

for checkinList in data:
	visCount = 0;
	for item in checkinList["items"]:
		if 'venue' not in item:
			continue

		if 'visibility' not in item:
			# print(f"Visibility not found for {item['id']} name {item['venue']['name']}")
			visCount += 1
			item['visibility'] = "private"
   
		if 'shout' not in item:
			# print(f"Visibility not found for {item['id']} name {item['venue']['name']}")
			item['shout'] = ""
   
		df = pd.concat([pd.DataFrame([[
      item['id'],
      item['createdAt'],
      item['type'],
      item['visibility'],
      item['timeZoneOffset'],
      item['venue']['id'],
      item['venue']['name'],
      item['venue']['url'],
      item['comments']['count'],
      item['shout']
      ]], columns=df.columns), df], ignore_index=True)

print(f"Total checkins without visibility: {visCount}")
print(df.shape)

Total checkins without visibility: 1000
(14265, 9)


In [48]:
print(df.loc[df['id']=='61180ce26f18ec40cccfcf73'])

                             id                   createdAt     type  \
13872  61180ce26f18ec40cccfcf73  2021-08-14 18:35:14.000000  checkin   

      visibility timeZoneOffset                   venueId  \
13872    private           -240  4c15352fa1010f4706a94d18   

                                venueName  \
13872  Bannerman Island (Pollepel Island)   

                                                venueURL commentsCount  
13872  https://foursquare.com/v/bannerman-island-poll...             0  


In [14]:
print(df.query('venueName.str.contains("Patent Pending")'))

                            id                   createdAt     type  \
5961  673a6ec59646fb66c8b13d2d  2024-11-17 22:31:33.000000  checkin   

        visibility timeZoneOffset                   venueId       venueName  \
5961  closeFriends           -300  59e63da08c35dc3e57ab5520  Patent Pending   

                                               venueURL commentsCount  
5961  https://foursquare.com/v/patent-pending/59e63d...             0  


In [17]:

# Get tips file
tipsFile = open('../foursquare-export/tips.json', 'r')
print(f"Reading tips file")
tipsObject = json.load(tipsFile)
tipsSetObject = tipsObject['items']

# get Venue Ratings 
ratingsFile = open('../foursquare-export/venueRatings.json', 'r')
print(f"Reading ratings file")
ratingsObject = json.load(ratingsFile)

venueLikes = ratingsObject['venueLikes']
venueDislikes = ratingsObject['venueDislikes']
venueOkays = ratingsObject['venueOkays']

photosFile = open('../foursquare-export/photos.json', 'r')
print(f"Reading photos file")
photosObject = json.load(photosFile)
photosSetObject = photosObject['items']

Reading tips file
Reading ratings file
Reading photos file


In [ ]:
venuesDf = pd.DataFrame(columns=[
    'id', # venue id
    'name', # venue name
    'url', # venue url
    'latitude',
    'longitude',
    'tipString', # tip text
    'tipCreatedAt',
    'tipId',
    'tipUrl', # tip canonicalUrl
    'tipViews', # tip viewCount
    'tipAgreeCount', # tip agreeCount
    'tipDisagreeCount', # tip disagreeCount
    'rating', # from ratings file, can be like, dislike, okay
    'imageSuffix', # from photos file, is the `suffix` field
    'imageWidth', # from photos file, is the `width` field
    'imageHeight', # from photos file, is the `height` field
    'imageId', # from photos file, is the `id` field
    'imageCreatedAt', # from photos file, is the `createdAt` field  
    'checkIns' # array of string checkin IDs.
])
# venuesDf.set_index('id', inplace=True)
for index, row in df.iterrows():
	venueRow = venuesDf.loc[venuesDf['id']==row['venueId']]
	if venueRow.empty:
		# print(f"Venue not found for {row['venueId']}")
		# continue
		venuesDf.loc[-1] = [
      row['venueId'], 
      row['venueName'],
      row["venueURL"],
      "", # latitude
      "", # longitude
      "", # tipString
      "", # tipCreatedAt
      "", # tipId
      "", # tipUrl
      "", # tipViews
      "", # tipAgreeCount
      "", # tipDisagreeCount
      "", # rating
      "", # imageSuffix
      "", # imageWidth
      "", # imageHeight
      "", # imageId
      "", # imageCreatedAt
      [row['id']] # checkIns
      ]  # adding a row
		venuesDf.index = venuesDf.index + 1  # shifting index
		venuesDf = venuesDf.sort_index()  # sorting by index
	else:
		if row["id"] == "61180ce26f18ec40cccfcf73":
			print('venueRow needs to be appended')
			print(venueRow)
		# add a new checkin to the series of checkins for this venue
		venuesDf.loc[venuesDf['id'] == row['venueId'], 'checkIns'] = venuesDf.loc[venuesDf['id'] == row['venueId'], 'checkIns'].apply(lambda x: x + [row['id']])


print('venuesDf.shape')
print(venuesDf.shape)
print('venuesDf.shape checkins')
print(venuesDf.iloc[100]["checkIns"])
# print(venuesDf.loc[venuesDf['id']=="56f3cb1a498e1d05e938aae5"])

venuesDf.shape
(5759, 19)
venuesDf.shape checkins
['61180ce26f18ec40cccfcf73']


In [53]:

print(venuesDf.loc[venuesDf['id']=="51e07c2c498e838526ddbf60"])

                            id                 name  \
5750  51e07c2c498e838526ddbf60  Gantry Park Landing   

                                                    url latitude longitude  \
5750  https://foursquare.com/v/gantry-park-landing/5...                      

     tipString tipCreatedAt tipId tipUrl tipViews tipAgreeCount  \
5750                                                              

     tipDisagreeCount rating imageSuffix imageWidth imageHeight imageId  \
5750                                                                      

     imageCreatedAt                                           checkIns  
5750                 [53c1fe9411d22a43d04b5f0c, 53fe6f6c498eba86e35...  


In [61]:
def addVenueRating(ratingSet, ratingType, venueDFSet):
	for ratingItem in ratingSet:
		venueRatingId = ratingItem['url'].split('/')[-1]
		venueRow = venueDFSet.loc[venueDFSet['id']==venueRatingId]
		if venueRow.empty:
			print(f"Venue not found for {venueRatingId}")
			venueDFSet.loc[-1] = [
				ratingItem['id'], 
				ratingItem['name'],
				ratingItem["url"],
				"", # latitude
				"", # longitude
				"", # tipString
				"", # tipCreatedAt
				"", # tipId
				"", # tipUrl
				"", # tipViews
				"", # tipAgreeCount
				"", # tipDisagreeCount
				"", # rating
				"", # imageSuffix
				"", # imageWidth
				"", # imageHeight
				"", # imageId
				"", # imageCreatedAt
				[] # checkIns
			]  # adding a row
			venueDFSet.index = venueDFSet.index + 1  # shifting index
			venueDFSet = venueDFSet.sort_index()  # sorting by index
		venueDFSet.loc[venueDFSet['id'] == venueRatingId, 'rating'] = ratingType   

addVenueRating(venueLikes, "like", venuesDf)
addVenueRating(venueDislikes, "dislike", venuesDf)
addVenueRating(venueOkays, "okay", venuesDf)

print(venuesDf.loc[venuesDf['id']=="4a74aadbf964a5201adf1fe3"])
print(venuesDf.loc[venuesDf['id']=="5d8bdc1895c3d900085a2a55"])

Venue not found for 5373864b498e35eab63257ea
Venue not found for 4846c5acf964a5206f501fe3
                            id               name  \
5735  4a74aadbf964a5201adf1fe3  Foodcellar Market   

                                                    url latitude longitude  \
5735  https://foursquare.com/v/foodcellar-market/4a7...                      

     tipString tipCreatedAt tipId tipUrl tipViews tipAgreeCount  \
5735                                                              

     tipDisagreeCount   rating imageSuffix imageWidth imageHeight imageId  \
5735                   dislike                                              

     imageCreatedAt                                           checkIns  
5735                 [53d28bb1498ea086564111e7, 53e578a9498e1192bec...  
                            id     name  \
1603  5d8bdc1895c3d900085a2a55  Zen Yai   

                                                    url latitude longitude  \
1603  https://foursquare.com/v/zen-yai/5d8bdc

In [71]:
for tip in tipsSetObject:
	tipVenueId = tip["venue"]["id"]
	venueRow = venuesDf.loc[venuesDf['id']==tipVenueId]
	if venueRow.empty:
		print(f"Venue not found for {tip['id']}")
		venuesDf.loc[-1] = [
			tip["venue"]["id"], 
			tip["venue"]["name"], 
			tip["venue"]["url"], 
			"", # latitude
			"", # longitude
			"", # tipString
			"", # tipCreatedAt
			"", # tipId
			"", # tipUrl
			"", # tipViews
			"", # tipAgreeCount
			"", # tipDisagreeCount
			"", # rating
			"", # imageSuffix
			"", # imageWidth
			"", # imageHeight
			"", # imageId
			"", # imageCreatedAt
			[] # checkIns
		]  # adding a row  
		venuesDf.index = venuesDf.index + 1  # shifting index
		venuesDf = venuesDf.sort_index()  # sorting by index
	# print(f"Venue found for {tipVenueId}")
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipString'] = tip["text"]
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipCreatedAt'] = tip["createdAt"]
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipId'] = tip["id"]
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipViews'] = tip["viewCount"]
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipAgreeCount'] = tip["agreeCount"]
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipDisagreeCount'] = tip["disagreeCount"] 
	venuesDf.loc[venuesDf['id'] == tipVenueId, 'tipUrl'] = tip["canonicalUrl"]  
 
print(venuesDf.loc[venuesDf['id']=="64dd65470e981a714e4c9f6c"])

Venue not found for 5c1280b00336930039275a9f
Venue not found for 5a2b3f00cad1b6792a7e0b1a
Venue not found for 5921b8be6a5950627d18dafb
Venue not found for 5781a77f498e613aff24c128
Venue not found for 4e41a72b15201b1cfb509da9
                            id               name  \
3066  64dd65470e981a714e4c9f6c  First Light Books   

                                                    url latitude longitude  \
3066  https://foursquare.com/v/first-light-books/64d...                      

                                              tipString  \
3066  Great bookstore, with drinks, snacks, and plen...   

                    tipCreatedAt                     tipId  \
3066  2024-10-15 23:55:00.000000  670f00d47d191f5820d40dda   

                                                 tipUrl tipViews  \
3066  https://foursquare.com/item/670f00d47d191f5820...       29   

     tipAgreeCount tipDisagreeCount rating imageSuffix imageWidth imageHeight  \
3066             0                0   like       

In [95]:
photosDf = pd.DataFrame(columns=[
    'id', 
    'venueId',
    'createdAt',
    'suffix',
    'width',
    'height',
])
for photoObj in photosSetObject:
	checkinId = "";
	if "swarmapp" in photoObj['relatedItemUrl']:
		checkinId = photoObj['relatedItemUrl'].split('/')[-1]
		# print(f"checkinId is {checkinId}")
		checkinRow = df.loc[df['id'] == checkinId]
		if checkinRow.empty:
			print(f"Checkin not found for {checkinId}")
			continue
		venueId = checkinRow['venueId'].values[0]
		#print(f"Checkin found at {venueId} from {checkinId} ")
		#print(checkinRow)
	else:
		venueId = photoObj['relatedItemUrl'].split('/')[-1]
	# print(f"VenueId found for {venueId}")
	photosDf.loc[-1] = [
			photoObj["id"],
			venueId,
			photoObj["createdAt"],
			photoObj["suffix"],
			photoObj["width"],
			photoObj["height"],
		]  # adding a row
	photosDf.index = photosDf.index + 1  # shifting index
	photosDf = photosDf.sort_index()  # sorting by index
 
	if not venuesDf.loc[venuesDf['id'] == venueId, 'imageSuffix'].empty:
		# print(f"Image already in place for {venueRatingId}")
		continue
	#print(f"Venue found for {venueId} from {checkinId}")
	venuesDf.loc[venuesDf['id'] == venueId, 'imageSuffix'] = photoObj["suffix"]
	venuesDf.loc[venuesDf['id'] == venueId, 'imageWidth'] = photoObj["width"]
	venuesDf.loc[venuesDf['id'] == venueId, 'imageHeight'] = photoObj["height"]
	venuesDf.loc[venuesDf['id'] == venueId, 'imageId'] = photoObj["suffix"]
	venuesDf.loc[venuesDf['id'] == venueId, 'imageCreatedAt'] = photoObj["createdAt"]
	#print(f"Venue enriched for {venueId}")
	#print(venuesDf.loc[venuesDf['id'] == venueId])

print('photosDf.shape')
print(photosDf.shape) 
#print(venuesDf.loc[venuesDf['id']=="4bfeb605ac03952163daa880"])
#print(venuesDf.loc[venuesDf['id']=="5781a77f498e613aff24c128"])
print(venuesDf.loc[venuesDf['id']=="591b5547018cbb3a2391fb59"])

Checkin not found for 591b5547018cbb3a2391fb59
photosDf.shape
(777, 6)
Empty DataFrame
Columns: [id, name, url, latitude, longitude, tipString, tipCreatedAt, tipId, tipUrl, tipViews, tipAgreeCount, tipDisagreeCount, rating, imageSuffix, imageWidth, imageHeight, imageId, imageCreatedAt, checkIns]
Index: []


In [ ]:
# %load_ext autoreload
%autoreload 2
import data_processing

config = dotenv_values('../.env')
apiKey = config['FSQ_API_KEY']

data_processing.get_place_details("59e63da08c35dc3e57ab5520", apiKey)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'fsq_id': '59e63da08c35dc3e57ab5520', 'categories': [{'id': 10032, 'name': 'Night Club', 'short_name': 'Night Club', 'plural_name': 'Night Clubs', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_', 'suffix': '.png'}}, {'id': 13009, 'name': 'Cocktail Bar', 'short_name': 'Cocktail', 'plural_name': 'Cocktail Bars', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_', 'suffix': '.png'}}, {'id': 13021, 'name': 'Speakeasy', 'short_name': 'Speakeasy', 'plural_name': 'Speakeasies', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/secretbar_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'Unsure', 'geocodes': {'drop_off': {'latitude': 40.745138, 'longitude': -73.990299}, 'main': {'latitude': 40.745324, 'longitude': -73.990221}, 'roof': {'latitude': 40.745324, 'longitude': -73.990221}}, 'link': '/v3/places/59e63da08c35dc3e

{'fsq_id': '59e63da08c35dc3e57ab5520',
 'categories': [{'id': 10032,
   'name': 'Night Club',
   'short_name': 'Night Club',
   'plural_name': 'Night Clubs',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
    'suffix': '.png'}},
  {'id': 13009,
   'name': 'Cocktail Bar',
   'short_name': 'Cocktail',
   'plural_name': 'Cocktail Bars',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_',
    'suffix': '.png'}},
  {'id': 13021,
   'name': 'Speakeasy',
   'short_name': 'Speakeasy',
   'plural_name': 'Speakeasies',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/secretbar_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'Unsure',
 'geocodes': {'drop_off': {'latitude': 40.745138, 'longitude': -73.990299},
  'main': {'latitude': 40.745324, 'longitude': -73.990221},
  'roof': {'latitude': 40.745324, 'longitude': -73.990221}},
 'link': '/v3/places/59e63da08c35dc3e57ab5520',
 'location': {'a